In [ ]:
import os
import sys
import yaml
import pandas as pd
import pickle

from network_wrangler import RoadwayNetwork, TransitNetwork, Scenario, WranglerLogger
from lasso import Parameters
from lasso.seflorida.seflorida_roadway import SEFloridaRoadwayNetwork

In [ ]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.INFO)

## I/O

In [ ]:
root_dir = "serpm/"
input_dir = os.path.join(root_dir, "data", "interim", "8-standard-format")
gtfs_input_dir = os.path.join(root_dir, "data", "interim", "6-transit-standard")
output_dir = os.path.join(root_dir, "data", "processed")
lasso_dir = "Lasso/"

In [ ]:
parameters = Parameters(lasso_base_dir = lasso_dir)

## Data Reads

In [ ]:
link_file = os.path.join(input_dir, 'links.json')
node_file = os.path.join(input_dir, 'nodes.geojson')
shape_file = os.path.join(input_dir, 'shapes.geojson')

roadway_net = SEFloridaRoadwayNetwork.read(
    link_filename = link_file, 
    node_filename = node_file, 
    shape_filename = shape_file
)

In [ ]:
transit_net = TransitNetwork.read(feed_path = gtfs_input_dir)

## Attribute the network

### number of lanes

In [ ]:
r_net = roadway_net.calculate_number_of_lanes(
    roadway_network = roadway_net,
    parameters = parameters,
    network_variable = "lanes"
)

r_net.links_df.lanes.value_counts()

In [ ]:
pd.crosstab(r_net.links_df.lanes, r_net.links_df.roadway)

### facility type

In [ ]:
pd.crosstab(r_net.links_df.roadway, r_net.links_df.oneWay)

In [ ]:
r_net = r_net.calculate_facility_type(
    roadway_network=r_net, parameters=parameters, network_variable="ftype"
)

r_net.links_df.ftype.value_counts()

In [ ]:
pd.crosstab(r_net.links_df.ftype, r_net.links_df.roadway)

### shorten roadname

In [ ]:
import re
def shorten_name(name):
    if type(name) == list:
        name_list = list(set(name))
        name_list = [
            re.sub(r"\W+", " ", c).replace("nan", "").strip(" ") for c in name_list
        ]
        name_new = " ".join(name_list).strip(" ")
    else:
        return name

In [ ]:
r_net.links_df["name"] = r_net.links_df["name"].apply(lambda x: shorten_name(x))

In [ ]:
r_net.links_df["name"].str.len().max()

### transit

In [ ]:
r_net = r_net.calculate_transit(
    roadway_network=r_net, parameters=parameters, network_variable="transit"
)
print(r_net.links_df.transit.value_counts())

## Create a Scenario

In [ ]:
base_scenario = {"road_net": r_net, "transit_net": transit_net}
working_scenario = Scenario.create_scenario(base_scenario = base_scenario)

## Write to disk

In [ ]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_01.pickle')
pickle.dump(working_scenario, open(working_scenario_filename, 'wb'))